# System Composition
- define clean method of setting system composition
- convert between units (e.g. g or mols)
- convert between components (e.g. endmembers, elements, oxides, etc.)

In [ ]:
import numpy as np
import pandas as pd
import scipy.linalg as lin

from thermoengine.model import SysComp

import matplotlib.pyplot as plt

In [ ]:
from thermoengine import core, phases, model, equilibrate

# Set composition manually

In [ ]:

oxide_comp = pd.DataFrame([{
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5,
}])
# Major oxides: SiO2  Al2O3   FeO* MgO  CaO  Na2O  K2O  H2O
oxide_comp     

In [ ]:
syscomp = SysComp(oxide_comp, components='oxides', units='wt')


In [ ]:
columns = syscomp.wt_comp().columns


In [ ]:
syscomp.wt_comp()

In [ ]:
syscomp.mol_comp()

In [ ]:
elem_comp = syscomp.mol_comp(components='elems')
elem_comp.values[0]
elem_comp.columns

In [ ]:
elem_comp

## Equilibrate system with this composition

In [ ]:
modelDB = model.Database(liq_mod='v1.0')

In [ ]:
Liquid = modelDB.get_phase('Liq')
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')
Spinel = modelDB.get_phase('SplS')
Opx = modelDB.get_phase('Opx')
RhomOx = modelDB.get_phase('Rhom')

The Berman model database provides the SWIM water model by default.  Instead, override that choice by instantiating the MELTS 1.0.2 water model directly.

In [ ]:
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)
phs_sys = [Liquid, Feldspar, Water, Quartz, Spinel, Opx, RhomOx]

In [ ]:
equil = equilibrate.Equilibrate(elem_comp.columns, phs_sys)

In [ ]:
T = 1050.0
P = 1750.0
state = equil.execute(T, P, bulk_comp=elem_comp.values[0], debug=0, stats=True)
state.print_state()

Pickup runs use previously computed state

## Default composition selection for major oxides

In [ ]:
comp = [77.5,12.5,.6,.03,.43,3.98,4.88,5.5]

syscomp = SysComp(comp, components='major_oxides')

In [ ]:
syscomp.elem_wts

In [ ]:
syscomp.endmember_wts

In [ ]:
syscomp.wt_comp()

In [ ]:
syscomp.wt_comp(normalize=True)

In [ ]:
syscomp.mol_comp(basis='formula', normalize=False)

In [ ]:
syscomp.mol_comp(basis='atomic')

In [ ]:
syscomp.wt_comp()

In [ ]:
syscomp.wt_comp().dot(syscomp.endmember_stoic)

In [ ]:
syscomp.endmember_stoic

In [ ]:
syscomp.mol_comp(components='oxides')

In [ ]:
elem_comp = syscomp.mol_comp(components='elems')
elem_comp

In [ ]:
elem_comp.columns

In [ ]:
elem_comp.values[0]

## Set nonzero oxides

In [ ]:
oxide_comp = pd.DataFrame([{
    'SiO2':  77.5, 
    'MgO': 0.1,
    'Al2O3': 12.5,
    'Na2O':   3.98, 
    'K2O':    4.88,
}])
oxide_comp

In [ ]:
syscomp = SysComp(oxide_comp, components='major_oxides')
syscomp.wt_comp()

## Set custom oxides

In [ ]:
oxide_comp = pd.DataFrame([{
    'SiO2':  77.5, 
    'MgO': 0.1,
    'Al2O3': 12.5,
    'Na2O':   3.98, 
    'K2O':    4.88,
}])
oxide_comp

In [ ]:
syscomp = SysComp(oxide_comp)
syscomp.wt_comp()

## Set custom non-oxide endmembers

In [ ]:
endmem_comp = pd.DataFrame([{
    'MgO': 0.1,
    'Al2O3': 12.5,
    'MgSiO3':  70.5, 
    'Na2O':   3.98, 
    'K2O':    4.88,
}])
endmem_comp

In [ ]:
syscomp = SysComp(endmem_comp, stoic={'MgSiO3': {'Mg':1, 'Si':1, 'O':3}})
syscomp.wt_comp()

## Override volatiles

In [ ]:

oxide_comp = pd.DataFrame([{
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5,
    'CO2':  0
}])
# Major oxides: SiO2  Al2O3   FeO* MgO  CaO  Na2O  K2O  H2O
oxide_comp  

In [ ]:
syscomp = SysComp(oxide_comp, H2O='none', CO2='none')
syscomp.wt_comp()